In [2]:
# You are allowed to use the following modules
import numpy as np
import matplotlib.pyplot as plt
from mountain_car import MountainCar
import pygame as pg
from pygame import gfxdraw

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Your code here
state_num=2
action_num=3
order=3
d=(order+1)**state_num
d_a=d*action_num

def scale(state_):
    a_ = np.array([(state_[0] + 1.2) / 1.7,(state_[1] + 0.07) / 0.14])
    return a_

def create_c(k_,n_):#number of states, order
    c_ = [[] for j in range(k_)]
    for i in range(k_):
        for j in range(int(np.power(n_ + 1,i))):
            for p in range(n_ + 1):
                for r in range(int(np.power(n_ + 1,k_- i - 1))):
                    c_[i].append(p)
    c_ = np.array(c_,dtype = np.int8)
    return c_
c=create_c(state_num,order)

def create_x_i(s_,action_index_): #for theta
    state = scale(s_)
    x_i=np.cos(np.pi*(np.matmul(state,c)))
    zero = np.zeros((1,d))
    X_ = np.empty(0)
    for i in range(action_num):
        if i == action_index_:
            X_ = np.append(X_,x_i)
        else:
            X_ = np.append(X_,zero)

    return X_

def create_x_i_v(s_): #for v_hat
    state = scale(s_)
    x_i_=np.cos(np.pi*(np.matmul(state,c)))
    return x_i_


def policy(st,theta):
    h = np.empty(0)
    for i in range(action_num):
        h = np.append(h,np.matmul(theta,create_x_i(st,i)))#theta * x(A_t,S_t
    exp = np.exp(h - np.max(h))
    soft_ = exp / np.sum(exp)

    return soft_

def creat_v_hat(s_in,w_in):
    x_ = create_x_i_v(s_in)
    v_ = np.matmul(w_in,x_)
    return v_ , x_

def nubla_ln_policy(s_,action_index_,theta_):
    x_i_ = create_x_i(s_,action_index_)
    soft_ = policy(s_,theta_)
    sum_ = np.zeros((1,len(x_i_)))
    for i in range(action_num):
        sum_ += soft_[i] * create_x_i(s_,i)
    nubla = x_i_ - sum_
    return nubla
    

In [5]:
def REINFORCE_with_Baseline(alpha_theta,alpha_w):
    w = np.zeros((1,d))
    theta = np.zeros((1,action_num * d))
    episode = 0
    step = np.empty(0)
    state_list_ = []
    step_per_episode_ = []
    reward_sum_ = []
    car2 = MountainCar()
    while episode != 20:
        episode += 1
        print("episode=_________________________________________________",episode)
        s = np.zeros((1,2))# states
        action = np.empty(0)
        s[0] = [(np.random.rand()*0.2)-0.6,0]# create the first state in [-.6 -0.4]
        car2.goal_reached = False
        t = 0
        aaa=1
        while not car2.goal_reached:
            soft_vector=policy(s[t],theta)
            action_index = np.random.choice(action_num,p=soft_vector)
            action = np.append(action,action_index)
            next_pos, next_vel, r_next, goal_reached = car2.move(s[t][0], s[t][1], action[t]-1)#-1 is for changing index to real action 
            s = np.append(s,[[next_pos,next_vel]],axis = 0)
            if t==aaa*10000:
                print(s[t + 1])
                print(soft_vector)
                aaa+=1
            
            t+=1
        T = t
        print("episode length=",T)
        for t_ in range(T):
            G = -1*(T-t_)
            v  , x_v= creat_v_hat(s[t_],w)
            delta = G - v
            
            w += alpha_w * delta * x_v
            theta += alpha_theta * delta * nubla_ln_policy(s[t_],action[t_],theta)
        step_per_episode_.append(T)
        state_list_.append(s)
        reward_sum_.append(-T)
    print("__")
    return state_list_,step_per_episode_ , reward_sum_ 

state_list_R,step_per_episode_R , reward_sum_R = REINFORCE_with_Baseline(0.0000095,0.0015)#t,w

episode=_________________________________________________ 1
[-0.86363183  0.03038487]
[0.33333333 0.33333333 0.33333333]
[ 0.21518454 -0.02103978]
[0.33333333 0.33333333 0.33333333]
[-0.64644692 -0.00603818]
[0.33333333 0.33333333 0.33333333]
[-0.33908864 -0.03044312]
[0.33333333 0.33333333 0.33333333]
[ 0.20542417 -0.01125268]
[0.33333333 0.33333333 0.33333333]
[-0.47539423  0.0376622 ]
[0.33333333 0.33333333 0.33333333]
[-0.45263051  0.02547417]
[0.33333333 0.33333333 0.33333333]
[-0.23302031  0.02821495]
[0.33333333 0.33333333 0.33333333]
episode length= 80765
episode=_________________________________________________ 2
[-0.81635992  0.04206104]
[9.99917564e-01 4.79586146e-05 3.44769649e-05]
[-0.90544495 -0.04715186]
[3.25866681e-33 1.00000000e+00 2.22481434e-39]
[ 0.29407429 -0.01063315]
[9.88102151e-27 1.00000000e+00 1.42196219e-14]
[-0.51231109  0.0474112 ]
[7.22604006e-06 6.93617407e-01 3.06375367e-01]
[-0.85149177  0.04191105]
[9.99921796e-01 4.15988841e-05 3.66047875e-05]
[-0.7

[-0.08619508  0.04244083]
[2.11059544e-06 4.21985720e-02 9.57799317e-01]
[-0.86635056  0.03999563]
[9.99152104e-01 1.88142721e-04 6.59753307e-04]
[-1.12392921 -0.03377973]
[8.69975788e-28 1.00000000e+00 1.30995497e-38]
[-0.7864382 -0.0516874]
[7.05273410e-33 1.00000000e+00 8.18082255e-36]
[ 0.20719728 -0.02043096]
[3.93335154e-20 9.99999999e-01 6.99168844e-10]
[-0.14279892  0.04395124]
[8.12571518e-08 4.43191926e-03 9.95567999e-01]
[-1.01327162  0.03139032]
[5.69911729e-09 2.98645899e-02 9.70135404e-01]
[-0.6114789 -0.0553681]
[1.48837330e-28 1.00000000e+00 4.83471233e-26]
[ 0.30253928 -0.00830686]
[1.24208736e-26 1.00000000e+00 1.00935058e-14]
[-0.2252587   0.04583206]
[1.52568017e-08 1.84354969e-01 8.15645015e-01]
[-0.85136322 -0.04952471]
[1.99224003e-33 1.00000000e+00 3.53258518e-38]
[ 0.2988816  -0.01024861]
[4.70469899e-27 1.00000000e+00 4.83336078e-15]
[-0.21238381  0.04682853]
[2.64315993e-08 3.84933718e-01 6.15066255e-01]
[-0.48292946  0.04825105]
[1.89220671e-07 9.45288592e-0

KeyboardInterrupt: 